In [ ]:
import os

import h5py
import numpy as np
from IPython.display import Video


def get_label(ans):
    try:
        ans = int(ans)
    except:
        print("Wrong Input")
        return False
    if ans not in [1, 2, 3]:
        print("Invalid option.")
        return False
    if ans == 1:
        return [0]
    elif ans == 2:
        return [1]
    else:
        return [0.5]


def create_human_label(
    video_dir, num_query=100, start_idx=None, width=1000, height=500
):
    print("START!")
    if start_idx:
        assert start_idx > 0, "you must input with video number (1, 2, 3, ...)"
        interval = range(start_idx - 1, num_query)
    else:
        interval = range(num_query)
    pref_labels = []
    for i in interval:
        label = False
        while not label:
            print(f"\nVideo {i + 1}")
            video_file = os.path.join(video_dir, f"idx{i}.mp4")
            display(
                Video(
                    video_file,
                    width=width,
                    height=height,
                    html_attributes="loop autoplay",
                )
            )
            reward = input(
                f"[{i + 1}/{num_query}] Put Preference (1 (left), 2 (right), 3 (equal)):  "
            ).strip()
            label = get_label(reward)
            if label:
                pref_labels.append(label[0])
    return np.asarray(pref_labels)

In [ ]:
env_pref_file = "../ant_labels/AntMaze_Medium-v4_pref.hdf5"
labels = create_human_label("../ant_labels/AntMaze_Medium-v4")

with h5py.File(env_pref_file, "a") as f:
    if "labels" in f:
        del f["labels"]
    f.create_dataset("labels", data=labels, chunks=True)